# What to do with a Flock of lambdas

* Library and talk by Andy Fundinger
* Ciemaar@fundinger.name
* Bloomberg Senior Engineer

## What is flock good for?

Usually I get that question at the end so, to save time I'm putting it at the beginning

* it's fun
* similar to a spreadsheet
* dealing with poorly defined mathematical systems; like taxes and roleplaying games

### Some normal functions

In [24]:
def func(x):
    return x+4

func = lambda x: x+4

const_four = lambda: 4 # a constant lambda

const_four()

4

### Some closures

In [25]:
ll       = ['PyCon', 'PyGotham', 'PyLondinium', 'PyTennessee', 'PyOhio']

item_two = lambda: ll[2]

item_two()

'PyLondinium'

In [26]:
ll[1], ll[2] = ll[2], ll[1]

item_two()

'PyGotham'

## Ok, enough of that, let's switch to dictionaries

We're going to make some closures and put them in the same dictionary that is inside them.

It's all references, so there's no particular problem here

In [27]:
invoice  = dict()
invoice['qty']       = lambda: 30
invoice['price']     = lambda: 100
invoice['sub_total'] = lambda: invoice['qty']() * invoice['price']()
invoice['tax_rate']  = lambda: .06625  # NJ Sales Tax
invoice['tax']       = lambda: invoice['sub_total']() * invoice['tax_rate']()
invoice['total']     = lambda: invoice['sub_total']() + invoice['tax']()

invoice['sub_total'](), invoice['tax'](), invoice['total']() 

(3000, 262.5, 3262.5)

In [28]:
invoice['tax']()

262.5

### But we're in New York so sales tax is 8.75%

In [29]:
invoice['tax_rate'] = lambda: .0875 # New York City Tax

invoice['tax'](), invoice['total']() 

(240.0, 3240.0)

# Fine, it's a neat trick, lets clean it up

* storing a constant should not require the creation of a constant lambda
* we know we want to call the function whenever we access it, so we'll build that in 
* we may have some caching eventually so let's be ready for that

In [30]:
import flock

invoice  = flock.FlockDict()
invoice['qty']       = 30
invoice['price']     = 100
invoice['tax_rate']  = .0875
invoice['sub_total'] = lambda: invoice['qty'] * invoice['price']
invoice['tax']       = lambda: invoice['sub_total'] * invoice['tax_rate']
invoice['total']     = lambda: invoice['sub_total'] + invoice['tax']

invoice['sub_total'], invoice['tax'], invoice['total']

(3000, 262.5, 3262.5)

## But if we're here, lets add some tricks

The aggregator lets us work across parallel maps

In [37]:
import flock.core

invoice  = flock.FlockDict()
invoice['qty']       = {1:2,  2:3,  3:5,  4:7, 5:11}
invoice['price_ea']  = {1:30, 2:60, 3:90, 4:2, 5:4 }
invoice['tax_rate']  = .0875

# Aggregator
invoice['price']     = flock.core.FlockAggregator([invoice['qty'],invoice['price_ea']], lambda x: x[0]*x[1])
invoice['sub_total'] = lambda: sum(pr for pr in invoice['price'].values())
invoice['tax']       = lambda: invoice['sub_total'] * invoice['tax_rate']
invoice['total']     = lambda: invoice['sub_total'] + invoice['tax']

invoice['sub_total'], invoice['tax'], invoice['total']

(748, 65.45, 813.45)

In [38]:
invoice['price'][3]

450

In [39]:
invoice['price_ea'][3] *= 1.05
invoice['price'][3], invoice['total']

(472.5, 837.91875)

In [40]:
invoice.shear()

OrderedDict([('price', {1: 60, 2: 180, 3: 472.5, 4: 14, 5: 44}),
             ('price_ea',
              OrderedDict([(1, 30), (2, 60), (3, 94.5), (4, 2), (5, 4)])),
             ('qty', OrderedDict([(1, 2), (2, 3), (3, 5), (4, 7), (5, 11)])),
             ('sub_total', 770.5),
             ('tax', 67.41874999999999),
             ('tax_rate', 0.0875),
             ('total', 837.91875)])

## GitHub etc.

* https://github.com/Ciemaar/flock
* Ciemaar@fundinger.name
* Follow us on Twitter: [@TechAtBloomberg](http://www.twitter.com/TechAtBloomberg)

## What is this good for?

* I'd love to hear what you suggest
* certain kinds of models
* dealing with poorly defined mathematical systems; like taxes and roleplaying games

In [34]:
from mythica.model import *
import yaml

char = FlockDict()
for key, value in yaml.load(open('dram.yaml')).items():
    char[key] = value

apply_rules(char)
char['points'].shear()

OrderedDict([('available',
              OrderedDict([('heroic', 14),
                           ('mental', -0.5),
                           ('physical', -8.0),
                           ('universal', 1.5)])),
             ('spent',
              OrderedDict([('heroic', 85),
                           ('mental', 14),
                           ('physical', 17),
                           ('universal', 8.5)])),
             ('total',
              OrderedDict([('heroic', 99),
                           ('mental', 13.5),
                           ('physical', 9.0),
                           ('universal', 10)]))])